Trước đây bạn đã huấn luyện Mạng nơ-ron 2 lớp (với một lớp ẩn duy nhất). Trong lab này, bạn sẽ xây dựng một mạng nơ-ron sâu với bao nhiêu lớp tùy thích!

- Trong notebook này, bạn sẽ thực hiện tất cả các hàm cần thiết để xây dựng một mạng nơ-ron sâu.
- Trong lab tiếp theo, bạn sẽ sử dụng các hàm này để xây dựng một mạng nơ-ron sâu cho phân loại ảnh.

**Sau lab này, bạn sẽ có thể:**
- Sử dụng các đơn vị phi tuyến tính như ReLU để cải thiện mô hình của bạn
- Xây dựng mạng nơ-ron sâu hơn (với nhiều hơn 1 lớp ẩn)
- Triển khai một lớp mạng nơ-ron dễ sử dụng

**Kí hiệu**:
- Chỉ số trên $[l]$ biểu thị số lượng được liên kết với lớp $l^{th}$. 
    - Ví dụ: $a^{[L]}$ là kích hoạt lớp $L^{th}$. $W^{[L]}$ và $b^{[L]}$ là các tham số lớp $L^{th}$.
- Chỉ số trên $(i)$ biểu thị số lượng được liên kết với ví dụ $i^{th}$. 
    - Ví dụ: $x^{(i)}$ là ví dụ huấn luyện $i^{th}$.
- Chỉ số dưới $i$ biểu thị mục nhập $i^{th}$ của một vectơ.
    - Ví dụ: $a^{[l]}_i$ biểu thị mục nhập $i^{th}$ của các lần kích hoạt lớp $l^{th}$.

Hãy bắt đầu!

## 1 - Thư viện

Đầu tiên, hãy nhập tất cả các thư viện mà chúng ta sẽ cần trong lab này.
- [numpy](www.numpy.org) là thư viện chính cho tính toán khoa học trong Python.
- [matplotlib](http://matplotlib.org) là một thư viện để vẽ đồ thị trong Python.
- dnn_utils cung cấp một số hàm cần thiết cho notebook này.
- testCases cung cấp một số trường hợp thử nghiệm để đánh giá tính đúng đắn của các hàm
- np.random.seed(1) được sử dụng để giữ cho tất cả các lệnh gọi hàm ngẫu nhiên nhất quán. Nó sẽ giúp chúng ta đánh giá công việc. Vui lòng không thay đổi seed.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v3 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # thiết lập kích thước mặc định của biểu đồ
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Sơ lược về Lab

Để xây dựng mạng nơ-ron của mình, chúng ta sẽ thực hiện một số "helper function" (hàm trợ giúp). Các hàm trợ giúp này sẽ được sử dụng trong lab tiếp theo để xây dựng mạng nơ-ron hai lớp và mạng nơ-ron L-lớp. Mỗi hàm trợ giúp nhỏ mà chúng ta thực hiện sẽ có hướng dẫn chi tiết hướng dẫn qua các bước cần thiết. Đây là bản sơ lược của lab này, chúng ta sẽ:

- Khởi tạo các tham số cho mạng hai lớp và cho mạng nơ-ron $ L $ -lớp. 
- Thực hiện mô-đun lan truyền xuôi (được hiển thị bằng màu tím trong hình bên dưới).
     - Hoàn thành phần TUYẾN TÍNH của bước lan truyền xuôi của một lớp (kết quả là $ Z ^ {[l]} $).
     - Chúng tôi cung cấp cho bạn hàm KÍCH HOẠT (relu/sigmoid).
     - Kết hợp hai bước trước đó thành một hàm forward [LINEAR->ACTIVATION] mới.
     - Xếp chồng hàm chuyển tiếp [LINEAR->RELU] L-1 lần (cho lớp 1 đến lớp L-1) và thêm [LINEAR->SIGMOID] vào cuối (cho lớp cuối cùng $ L $). Điều này cung cấp cho bạn hàm L_model_osystem mới.
- Tính loss (mất mát).
- Thực hiện mô-đun lan truyền ngược (ký hiệu màu đỏ trong hình bên dưới).
    - Hoàn thành phần LINEAR của bước lan truyền ngược của một lớp.
    - Chúng tôi cung cấp cho bạn gradient của hàm ACTIVATE (relu_backward/sigmoid_backward)
    - Combine the previous two steps into a new [LINEAR->ACTIVATION] backward function. Kết hợp hai bước trước đó thành hàm backward [LINEAR->ACTIVATION] mới.
    - Xếp chồng [LINEAR->RELU] lùi L-1 lần và thêm [LINEAR->SIGMOID] lùi vào hàm L_model_backward mới
- Cuối cùng là cập nhật các tham số.

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center> **Figure 1**</center></caption><br>


**Lưu ý** rằng đối với mỗi hàm forward, có một hàm backward tương ứng. Đó là lý do tại sao ở mỗi bước của mô-đun chuyển tiếp, chúng ta sẽ lưu trữ một số giá trị trong cache. Các giá trị được lưu trong cache rất hữu ích cho việc tính toán gradient. Sau đó, trong mô-đun backpropagation, chúng ta sẽ sử dụng cache để tính toán các gradient. Lab này sẽ chỉ cho bạn chính xác cách thực hiện từng bước này.

## 3 - Khởi tạo

Chúng ta sẽ viết hai hàm trợ giúp khởi tạo các tham số cho mô hình của mình. Hàm đầu tiên sẽ được sử dụng để khởi tạo các tham số cho mô hình 2 lớp. Hàm thứ hai sẽ tổng quát quá trình khởi tạo này thành $ L $ lớp.

### 3.1 - Mạng nơ-ron 2 lớp

**Task 1**: Tạo và khởi tạo các tham số của mạng nơ-ron 2 lớp.

**Hướng dẫn**:
- Cấu trúc của mô hình là: *LINEAR -> RELU -> LINEAR -> SIGMOID*.
-Sử dụng khởi tạo ngẫu nhiên cho các ma trận trọng số. Sử dụng `np.random.randn(shape)*0.01` với shape chính xác.
- Sử dụng khởi tạo bằng không cho các độ lệch. Sử dụng `np.zeros(shape)`.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Đối số:
    n_x -- kích thước lớp đầu vào
    n_h -- kích thước lớp ẩn
    n_y -- kích thước lớp đầu ra
    
    Trả về:
    parameters -- dictionary của python chứa các tham số:
                    W1 -- ma trận trọng số có shape (n_h, n_x)
                    b1 -- vectơ bias có shape (n_h, 1)
                    W2 -- ma trận trọng số có shape (n_y, n_h)
                    b2 -- vectơ bias có shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 4 dòng code)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [ ]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]


**Kỳ vọng đầu ra**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td> [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]] </td> 
  </tr>

  <tr>
    <td> **b1**</td>
    <td>[[ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.01744812 -0.00761207]]</td>
  </tr>
  
  <tr>
    <td> **b2** </td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 3.2 - Mạng nơ-ron L lớp

Việc khởi tạo cho một mạng nơ-ron L lớp sâu hơn phức tạp hơn vì có nhiều ma trận trọng số và vectơ bias hơn. Khi hoàn thành `initialize_parameters_deep`, bạn nên đảm bảo rằng kích thước của bạn khớp với mỗi lớp. Nhớ lại rằng $ n ^ {[l]} $ là số đơn vị trong lớp $ l $. Vì vậy, ví dụ: nếu kích thước của đầu vào $ X $ của chúng ta là $ (12288, 209) $ (với $ m = 209 $ chẳng hạn) thì:

<table style="width:100%">


    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
   
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    
   <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    
    
   <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>

</table>

Hãy nhớ rằng khi chúng ta tính $ W X + b $ trong python, nó sẽ thực hiện broadcasting (truyền phát). Ví dụ, nếu:

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Thì $WX + b$ sẽ là:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**Task 2**: Thực hiện khởi tạo cho Mạng nơ-ron L lớp.

**Hướng dẫn**:
- Cấu trúc của mô hình là *[LINEAR->RELU]$\times $(L-1) -> LINEAR -> SIGMOID *. Tức là, nó có $ L-1 $ lớp sử dụng hàm kích hoạt ReLU, tiếp theo là lớp đầu ra có hàm kích hoạt sigmoid.
- Sử dụng khởi tạo ngẫu nhiên cho các ma trận trọng số. Sử dụng `np.random.rand(shape)*0.01`.
- Sử dụng khởi tạo thành 0 cho các bias. Sử dụng `np.zeros(shape)`.
- Chúng ta sẽ lưu trữ $ n ^ {[l]} $, số lượng đơn vị trong các lớp khác nhau trong biến `layer_dims`. Ví dụ: `layer_dims` cho "Mô hình phân loại dữ liệu phẳng" từ lab trước sẽ là [2,4,1]: Có hai đầu vào, một lớp ẩn với 4 đơn vị ẩn và một lớp đầu ra có 1 đơn vị đầu ra . Như vậy có nghĩa là shape của `W1` là (4,2), `b1` là (4,1), `W2` là (1,4) và `b2` là (1,1). Bây giờ bạn sẽ tổng quát điều này thành $ L $ lớp!
- Đây là cách thực hiện cho $ L = 1 $ (mạng nơ-ron 1 lớp). Nó sẽ truyền cảm hứng cho bạn để triển khai trường hợp chung (mạng nơ-ron L lớp).
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [ ]:
def initialize_parameters_deep(layer_dims):
    """
    Đối số:
    layer_dims -- mảng (list) của python chứa các chiều của từng lớp trong mạng
    
    Trả về:
    parameters -- dictionary của python chứa các tham số "W1", "b1", ..., "WL", "bL":
                    Wl -- ma trận trọng số có shape (layer_dims[l], layer_dims[l-1])
                    bl -- vectơ bias có shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # số lớp trong mạng

    for l in range(1, L):
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        ### KẾT THÚC CODE Ở ĐÂY ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [ ]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Kỳ vọng đầu ra**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>**b1** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2** </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>**b2** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4 - Mô-đun lan truyền xuôi

### 4.1 - Truyền xuôi tuyến tính (Linear Forward)
Giờ bạn đã khởi tạo các tham số của mình, hãy thực hiện mô-đun lan truyền xuôi. Bạn sẽ bắt đầu bằng cách thực hiện một số hàm cơ bản mà bạn sẽ sử dụng sau này khi thực hiện mô hình. Hãy hoàn thành 3 hàm theo thứ tự sau:

- LINEAR
- LINEAR -> ACTIVATION trong đó ACTIVATION hoặc là ReLU hoặc Sigmoid. 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (toàn mô hình)

Mô-đun truyền xuôi tuyến tính (được vector hóa trên tất cả các ví dụ) tính các phương trình sau:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

trong đó $A^{[0]} = X$. 

**Task 3**: Xây dựng phần tuyến tính của lan truyền xuôi.

**Nhắc nhở**:
Biểu diễn toán học của đơn vị này là $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$. Bạn cũng có thể thấy `np.dot()` hữu ích. Nếu chiều không khớp, việc in `W.shape` có thể hữu ích.

In [ ]:
def linear_forward(A, W, b):
    """
    Triển khai phần truyền xuôi tuyến tính của một lớp.

    Đối số:
    A -- các kích hoạt từ lớp trước đó (hoặc dữ liệu đầu vào): (kích thước lớp trước đó, số ví dụ)
    W -- ma trận trọng số: mảng numpy có shape (kích thước lớp hiện tại, kích thước lớp trước đó)
    b -- vectơ bias, mảng numpy có shape (kích thước lớp hiện tại, 1)

    Trả về:
    Z -- đầu vào của hàm kích hoạt, cũng gọi là tham số tiền kích hoạt
    cache -- dictionary của python chứa  "A", "W" và "b" ; được lưu trữ để tính toán truyền ngược hiệu quả
    """
    
    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 1 dòng code)
    Z = np.dot(W,A)+b
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**Kỳ vọng đầu ra**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4.2 - Truyền xuôi kích hoạt tuyến tính (Linear-Activation Forward)

Trong notebook này, chúng ta sẽ sử dụng 2 hàm kích hoạt:

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. Chúng tôi đã cung cấp cho bạn hàm `sigmoid`. Hàm này trả về **hai** mục: giá trị kích hoạt "`a`" và "`cache`" có chứa "`Z` "(đó là những gì chúng ta sẽ đưa vào hàm truyền ngược tương ứng). Để sử dụng nó, bạn chỉ cần gọi:
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: Công thức toán học cho ReLu là $A = RELU(Z) = max(0, Z)$. Chúng tôi đã cung cấp cho bạn hàm `relu`. Hàm này trả về **hai** mục: giá trị kích hoạt "`A` " và " `cache`" có chứa "`Z` "(đó là những gì chúng ta sẽ đưa vào hàm truyền ngược tương ứng). Để sử dụng nó, bạn chỉ cần gọi:
``` python
A, activation_cache = relu(Z)
```

Để thuận tiện hơn, chúng ta sẽ nhóm hai hàm (Tuyến tính và Kích hoạt) thành một hàm (TUYẾN TÍNH->KÍCH HOẠT). Do đó, chúng ta sẽ triển khai một hàm thực hiện bước truyền xuôi TUYẾN TÍNH, theo sau là bước truyền xuôi KÍCH HOẠT.

**Task 4**: Thực hiện truyền xuôi của lớp *LINEAR->ACTIVATION*. Quan hệ toán học là: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ trong đó kích hoạt "g" có thể là sigmoid() hoặc relu(). Sử dụng linear_forward() và hàm kích hoạt chính xác.

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Triển khai truyền xuôi cho lớp LINEAR->ACTIVATION

    Đối số:
    A_prev -- các kích hoạt từ lớp trước đó (hoặc dữ liệu đầu vào): (kích thước lớp trước đó, số ví dụ)
    W -- ma trận trọng số: mảng numpy có shape (kích thước lớp hiện tại, kích thước lớp trước đó)
    b -- vectơ bias, mảng numpy có shape (kích thước lớp hiện tại, 1)
    activation -- hàm kích hoạt dùng trong lớp này, được lưu trữ thành xâu văn bản: "sigmoid" hoặc "relu"

    Trả về:
    A -- đầu ra của hàm kích hoạt, cũng gọi là giá trị hậu kích hoạt
    cache -- dictionary của python chứa "linear_cache" và "activation_cache";
             được lưu trữ để tính toán truyền ngược hiệu quả
    """
    
    if activation == "sigmoid":
        # Đầu vào: "A_prev, W, b". Đầu ra: "A, activation_cache".
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### KẾT THÚC CODE Ở ĐÂY ###
    
    elif activation == "relu":
        # Đầu vào: "A_prev, W, b". Đầu ra: "A, activation_cache".
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### KẾT THÚC CODE Ở ĐÂY ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [ ]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Kỳ vọng đầu ra**:
       
<table style="width:35%">
  <tr>
    <td> **With sigmoid: A ** </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> **With ReLU: A ** </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>


**Lưu ý**: Trong deep learning, phép tính "[LINEAR-> ACTIVATION]" được tính là một lớp trong mạng nơ-ron chứ không phải 2 lớp.

### d) Mô hình L-lớp

Để thuận tiện hơn nữa khi triển khai Mạng nơ-ron $ L $-lớp, bạn sẽ cần một hàm sao chép cái trước đó (`linear_activation_osystem` với RELU) $ L-1 $ lần, sau đó sử dụng `linear_activation_forward` với SIGMOID.

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **Hình 2** : mô hình *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* </center></caption><br>

**Task 5**: Thực hiện lan truyền xuôi của mô hình trên.

**Hướng dẫn**: Trong đoạn code dưới đây, biến `AL` sẽ biểu thị $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. (Điều này đôi khi còn được gọi là `Y mũ`, tức là $\hat{Y}$.) 

**Lời khuyên**:
- Sử dụng các hàm bạn đã viết trước đó
- Sử dụng vòng lặp for để sao chép [LINEAR-> RELU] (L-1) lần
- Đừng quên theo dõi các cache trong list "caches". Để thêm giá trị mới `c` vào `list`, bạn có thể sử dụng `list.append(c)`.

In [ ]:
# HÀM PHÂN BẬC: L_model_forward

def L_model_forward(X, parameters):
    """
    Triển khai truyền xuôi cho tính toán [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID
    
    Đối số:
    X -- dữ liệu, mảng numpy có shape (kích thước đầu vào, số ví dụ)
    parameters -- đầu ra của initialize_parameters_deep()
    
    Trả về:
    AL -- giá trị hậu kích hoạt trước
    caches -- danh sách caches có chứa:
                mọi cache của linear_relu_forward() (có L-1 trong số đó, được lập chỉ mục từ 0 tới L-2)
               cache của linear_sigmoid_forward() (chỉ có một, được lập chỉ mục L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # số lớp trong mạng nơ-ron
    
    # Triển khai [LINEAR -> RELU]*(L-1). Thêm "cache" vào danh sách "caches".
    for l in range(1, L):
        A_prev = A 
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        W=parameters['W' + str(l)]
        b=parameters['b' + str(l)]
        A, cache = linear_activation_forward(A_prev, W, b, "relu")
        caches.append(cache)
        
        ### KẾT THÚC CODE Ở ĐÂY ###
    
    # Triển khai LINEAR -> SIGMOID. Thêm "cache" vào danh sách "caches".
    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], "sigmoid")
    caches.append(cache)

    ### KẾT THÚC CODE Ở ĐÂY ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [ ]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


<table style="width:50%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 3 </td> 
  </tr>
</table>

Tuyệt! Bây giờ bạn có một lan truyền xuôi đầy đủ lấy đầu vào X và xuất ra một vectơ hàng  $A^{[L]}$ chứa các dự đoán. Nó cũng ghi lại tất cả các giá trị trung gian trong "caches". Với $A^{[L]}$, chúng ta có thể tính toán chi phí (cost) của dự đoán.

## 5 - Hàm chi phí (Cost function)

Bây giờ bạn sẽ thực hiện lan truyền xuôi và truyền ngược. Chúng ta cần tính toán chi phí để kiểm tra xem mô hình có thực sự đang học hay không.

**Task 6**: Tính cross-entropy cost $J$ sử dụng công thức sau: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [ ]:
# HÀM PHÂN BẬC: compute_cost

def compute_cost(AL, Y):
    """
    Triển khai hàm chi phí được xác định bởi phương trình (7).

    Đối số:
    AL -- vectơ xác suất tưng ứng với nhãn dự đoán, shape (1, số ví dụ)
    Y -- vectơ true "label" (ví dụ: chứa 0 nếu không phải mèo, 1 nếu là mèo), shape (1, số ví dụ)

    Trả về:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Tính mất mát từ aL và y.
    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 1 dòng code)
    cost = -(1/m)*np.sum(np.multiply(Y,np.log(AL))+np.multiply(1-Y,np.log(1-AL)))
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    cost = np.squeeze(cost)      # Để đảm bảo hình dạng của chi phí như những gì chúng ta mong đợi (ví dụ: biến [[17]] thành 17).
    assert(cost.shape == ())
    
    return cost

In [ ]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


**Kỳ vọng đầu ra**:

<table>

    <tr>
    <td>**cost** </td>
    <td> 0.41493159961539694</td> 
    </tr>
</table>

## 6 - Mô-đun lan truyền ngược

Giống với lan truyền xuôi, chúng ta sẽ triển khai các hàm trợ giúp cho lan truyền ngược. Hãy nhớ rằng lan truyền ngược được sử dụng để tính toán gradient của hàm mất mát liên quan đến các tham số.

**Nhắc nhở**: 
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center> **Hình 3** : Lan truyền xuôi và lan truyền ngược cho *LINEAR->RELU->LINEAR->SIGMOID* <br> *Các khối màu tím đại diện cho lan truyền xuôi và các khối màu đỏ đại diện cho lan truyền ngược.*  </center></caption>

<!-- 
For those of you who are expert in calculus (you don't need to be to do this assignment), the chain rule of calculus can be used to derive the derivative of the loss $\mathcal{L}$ with respect to $z^{[1]}$ in a 2-layer network as follows:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

In order to calculate the gradient $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$, you use the previous chain rule and you do $dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$. During the backpropagation, at each step you multiply your current gradient by the gradient corresponding to the specific layer to get the gradient you wanted.

Equivalently, in order to calculate the gradient $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$, you use the previous chain rule and you do $db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$.

This is why we talk about **backpropagation**.
!-->

Bây giờ, tương tự như lan truyền xuôi, chúng ta sẽ xây dựng lan truyền ngược theo 3 bước::
- LINEAR ngược
- LINEAR -> ACTIVATION ngược nơi ACTIVATION tính đạo hàm của kích hoạt ReLU hoặc sigmoid
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID ngược (toàn bộ mô hình)

### 6.1 - Truyền ngược tuyến tính (Linear backward)

Đối với lớp $l$, phần tuyến tính là: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (tiếp theo là kích hoạt).

Giả sử bạn đã tính đạo hàm $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. Bạn muốn nhận $(dW^{[l]}, db^{[l]} dA^{[l-1]})$.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

Ba đầu ra $(dW^{[l]}, db^{[l]}, dA^{[l]})$ được tính bằng đầu vào $dZ^{[l]}$. Đây là công thức bạn cần:
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


**Task 7**: Sử dụng 3 công thức ở trên để thực hiện linear_backward().

In [ ]:
def linear_backward(dZ, cache):
    """
    Triển khai phần tuyến tính của lan truyền ngược cho một lớp duy nhất (lớp l)

    Đối số:
    dZ -- Gradient của chi phí liên quan tới đầu ra tuyến tính (của lớp l hiện tại)
    cache -- tuple của các giá trị (A_prev, W, b) từ lan truyền xuôi ở lớp hiện tại

    Trả về:
    dA_prev -- Gradient của chi phí liên quan tới kích hoạt (của lớp l-1 trước), có cùng shape với A_prev
    dW -- Gradient của chi phí liên quan tới W (current layer l), có cùng shape với W
    db -- Gradient của chi phí liên quan tới b (current layer l), có cùng shape với b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 3 dòng code)
    dW = (1/m)*np.dot(dZ,np.transpose(A_prev))
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(np.transpose(W),dZ)
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
# Thiết lập một số đầu vào kiểm tra
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


**Kỳ vọng đầu ra**: 

<table style="width:90%">
  <tr>
    <td> **dA_prev** </td>
    <td > [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]] </td> 
  </tr> 
  
    <tr>
        <td> **dW** </td>
        <td > [[-0.10076895  1.40685096  1.64992505]] </td> 
    </tr> 
  
    <tr>
        <td> **db** </td>
        <td> [[ 0.50629448]] </td> 
    </tr> 
    
</table>



### 6.2 - Truyền ngược kích hoạt tuyến tính (Linear-Activation backward)

Tiếp theo, chúng ta sẽ tạo một hàm kết hợp 2 hàm trợ giúp: **`linear_backward`** và bước truyền ngược để kích hoạt **`linear_activation_backward`**.

Để giúp bạn triển khai `linear_activation_backward`, chúng tôi đã cung cấp 2 hàm truyền ngược:
- **`sigmoid_backward`**: Thực hiện truyền ngược cho đơn vị SIGMOID. Bạn có thể gọi nó như sau: 

```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward`**: Thực hiện truyền ngược cho đơn vị RELU. Bạn có thể gọi nó như sau:

```python
dZ = relu_backward(dA, activation_cache)
```

Nếu $g(.)$ là hàm kích hoạt, 
`sigmoid_backward` và `relu_backward` sẽ tính $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$.  

**Task 8**: Thực hiện lan truyền ngược cho lớp *LINEAR->ACTIVATION*.

In [ ]:
def linear_activation_backward(dA, cache, activation):
    """
    Triển khai lan truyền ngược cho lớp LINEAR->ACTIVATION.
    
    Đối số:
    dA -- gradient hậu kích hoạt cho lớp l hiện tại
    cache -- tuple của các giá trị (linear_cache, activation_cache) mà chúng ta lưu trữ để tính toán lan truyền ngược một cách hiệu quả
    activation -- kích hoạt dùng trong lớp này được lưu trữ thành xâu văn bản: "sigmoid" hoặc "relu"
    
    Trả về:
    dA_prev -- Gradient của chi phí liên quan tới kích hoạt (của lớp l-1 trước), có cùng shape với A_prev
    dW -- Gradient của chi phí liên quan tới W (current layer l), có cùng shape với W
    db -- Gradient của chi phí liên quan tới b (current layer l), có cùng shape với b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        ### KẾT THÚC CODE Ở ĐÂY ###
        
    elif activation == "sigmoid":
        ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 2 dòng code)
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        ### KẾT THÚC CODE Ở ĐÂY ###
    
    return dA_prev, dW, db

In [ ]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Kỳ vọng đầu ra với sigmoid:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 

  </tr> 
  
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr> 
  
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>



**Kỳ vọng đầu ra với relu:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 

  </tr> 
  
    <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
  
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>



### 6.3 - Truyền ngược L-mô hình

Bây giờ chúng ta sẽ thực hiện hàm truyền ngược cho toàn mạng. Nhớ lại rằng khi triển khai hàm `L_model_osystem`, ở mỗi lần lặp lại, chúng ta đã lưu trữ một cache chứa (X, W, b và z). Trong mô-đun lan truyền ngược, chúng ta sẽ sử dụng các biến đó để tính gradient. Do đó, trong hàm `L_model_backward`, chúng ta sẽ lặp lại tất cả các lớp ẩn truyền ngược, bắt đầu từ lớp $ L $. Ở mỗi bước, chúng ta sẽ sử dụng các giá trị được lưu trong bộ nhớ cache cho lớp $ l $ để sao chép thông qua lớp $ l $. Hình 5 dưới đây cho thấy tính ngược (Backward pass).


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **Hình 5** : Backward pass  </center></caption>

** Khởi tạo truyền ngược**:
Để truyền ngược thông qua mạng này, chúng ta biết đầu ra là
$A^{[L]} = \sigma(Z^{[L]})$. Do đó code của bạn cần tính `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$.
Để làm như vậy, hãy sử dụng công thức này (được suy ra bằng phép tính mà bạn không cần kiến ​​thức chuyên sâu):
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # đạo hàm của chi phí với AL
```

Sau đó, chúng ta có thể sử dụng gradient sau kích hoạt `dAL` này để tiếp tục truyền ngược. Như trong Hình 5, giờ chúng ta có thể nhập `dAL` vào hàm truyền ngược LINEAR-> SIGMOID mà các bạn đã triển khai (sẽ sử dụng các giá trị được lưu trong bộ nhớ cache được lưu trữ bởi hàm L_model_osystem). Sau đó, chúng ta sẽ phải sử dụng vòng lặp `for` để lặp qua tất cả các lớp khác bằng cách sử dụng hàm lùi LINEAR->RELU. Nên lưu trữ từng dA, dW và db trong dictionary grads. Để làm như vậy, hãy sử dụng công thức sau:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

Ví dụ: đối với $l=3$ điều này sẽ lưu trữ $dW^{[l]}$ trong `grads["dW3"]`.

**Task 9**: Thực hiện truyền ngược cho mô hình *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID*.

In [ ]:
def L_model_backward(AL, Y, caches):
    """
    Triển khai lan truyền ngược cho nhóm [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID 
    
    Đối số:
    AL -- vectơ xác suất, đầu ra của lan truyền ngược (L_model_forward())
    Y -- vectơ true "label" (chứa 0 nếu không phải mèo, 1 nếu là mèo)
    caches -- danh sách caches chứa:
                mọi cache của linear_activation_forward() với "relu" (là caches[l], cho l trong range(L-1) chẳng hạn: l = 0...L-2)
                cache của linear_activation_forward() với "sigmoid" (là caches[L-1])
    
    Trả về:
    grads -- dictionary với gradient
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # số lớp
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # sau dòng này, Y có cùng shape với AL
    
    # Khởi tạo truyền ngược
    ### BẮT ĐẦU CODE Ở ĐÂY ### (1 dòng code)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # đạo hàm của chi phí với AL
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    # Gradient lớp thứ l (SIGMOID -> LINEAR). Đầu vào: "AL, Y, caches". Đầu ra: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### BẮT ĐẦU CODE Ở ĐÂY ### (khoảng 2 dòng)
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    ### KẾT THÚC CODE Ở ĐÂY ###
    
    for l in reversed(range(L-1)):
        # lớp thứ l: (RELU -> LINEAR) gradient.
        # Đầu vào: "grads["dA" + str(l + 2)], caches". Đầu ra: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### BẮT ĐẦU CODE Ở ĐÂY ### (khoảng 5 dòng)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### KẾT THÚC CODE Ở ĐÂY ###

    return grads

In [ ]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


**Kỳ vọng đầu ra**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
  
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td> 

  </tr> 
</table>



### 6.4 - Cập nhật Tham số

Trong phần này, chúng ta sẽ cập nhật các tham số của mô hình, sử dụng gradient descent:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

trong đó $\alpha$ là learning rate. Sau khi tính toán các tham số đã cập nhật, hãy lưu trữ chúng trong dictionary tham số.

**Task 10**: Triển khai `update_parameters()` để cập nhật các tham số của bạn sử dụng gradient descent.

**Hướng dẫn**:
Cập nhật các tham số bằng cách sử dụng gradient descent trên mỗi $W^{[l]}$ và $b^{[l]}$ cho $l = 1, 2, ..., L$. 


In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Cập nhật tham số sử dụng gradient descent
    
    Đối số:
    parameters -- dictionary của python chứa các tham số 
    grads -- dictionary của python chứa gradient, đầu ra của L_model_backward
    
    Trả về:
    parameters -- dictionary của python chứa các tham số đã cập nhật 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # số lớp trong mạng nơ-ron

    # Quy tắc cập nhật cho từng tham số. Sử dụng vòng lặp for.
    ### BẮT ĐẦU CODE Ở ĐÂY ### (≈ 3 dòng code)
    for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db" + str(l+1)]
    ### KẾT THÚC CODE Ở ĐÂY ###
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**Kỳ vọng đầu ra**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
  
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
  
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>



## 7 - Tổng kết

Chúc mừng bạn đã triển khai tất cả các hàm cần thiết để xây dựng một mạng nơ-ron sâu!

Chúng tôi biết đây là một lab khá dài nhưng về sau mọi thứ sẽ tốt hơn. Phần tiếp theo của lab sẽ dễ dàng hơn.

Trong lab tiếp theo, bạn sẽ kết hợp tất cả những thứ này lại với nhau để tạo ra 2 mô hình:
- Mạng nơ-ron 2 lớp
- Mạng nơ-ron L lớp

Trên thực tế, bạn sẽ sử dụng những mô hình này để phân loại hình ảnh là mèo và không phải mèo!